# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [3]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory ‘data’: File exists


# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

> My name is Anand Patel, and I am an aerospace engineer with a passion for art, programming, and visualizations. I live in Houston, TX and I work for NASA Johnson Space Center in the Virtual Reality space. I am in my 4th semester of MIDS, and I am currently only taking w261. I would like this course to help me gain applied experience in Spark & Hadoop, and to give me a deeper understanding of ML algorithms from w207 and in recommender systems. I want to transition from my current industry to a career in data science, so I hope this course can prepare me for how ML is applied in practice at the companies I will be applying for later this year.

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> In supervised machine learning, the mean squared test error describes the error between our actual values and our predictions. The mean squared test error can be decomposed into two sources of error, described as the reducible error and the irreducible error, stemming from the noise intrinsic to our data. As the names suggest, we can reduce the reducible error, through our model and parameters, but we cannot reduce the irreducible error. The test error is decomposed as follows (source ISL, eqn 2.3):
>
> $E(Y -  \hat Y)^2 = [f(x) - \hat f(X)]^2 + Var(\epsilon)$
> 
> $[f(x) - \hat f(X)]^2$: Reducible Error
> 
> $Var(\epsilon)$: Irreducible Error
>
> 
> The reducible error is further decomposed into the bias and variance terms (source ISL, eqn 2.7):
>
> $E(Y -  \hat Y)^2 = Var(\hat f(x_0)) + [Bias(\hat f(x_0))]^2 + Var(\epsilon)$
>
> **The bias-variance tradeoff is the property of a model that the variance of the parameter estimates across samples can be reduced by increasing the bias in the estimated parameters. The bias–variance problem is the conflict in trying to simultaneously minimize these two sources of error that prevent supervised learning algorithms from generalizing beyond their training set (source Week 1 slides). Minimizing these two sources of error will minimize the reducible error in our problem.**
>
> The bias error comes from modeling, and high bias means that our model underfits our training data. A low bias model means our training data is captured well by our model. A more complex model will have lower bias.
>
>
> The variance is an error that comes from how sensitive the model is to small fluctuations or changes in the training data. A high variance means the algorithm is likely overfitting on the training data and modeling the noise. This will hinder the model when it comes to generalizing to new data. A more simple model will have lower variance. 
>
> In Machine Learning, the best models will neither overfit (high variance, low bias) nor underfit (low variance, high bias). They will strike a balance finding the model of appropriate complexity that minimizes both variance and bias in the model. This can be difficult since the data scientist does not directly observe the bias and variance of their models, and these two sources of error are generally in competition. There are bootstrapping techniques to decompose error into bias and variance, but not the irreducible error. 
>
> Sources:
> - Intro to Statistical Learning
> - Week 1 slides w261
> - https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote12.html


# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ Capitalization handling in the tokenizer yields different results for word frequency count. Let's say tokenizer 1 treats lowercase and uppercase as the same; then 'alice' would be counted twice. If tokenizer 2 treats lowercase and uppercase differently; then 'alice' would be counted once and 'Alice' would also be counted once seperately. These yield two different word frequencies. As a result, the likelihood of 'alice' would be different in these two implementations.

> __b)__ `['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']`

In [5]:
# Part C - Fill in the regular expression
# keep only lower case letters a through z
# tested with: https://regex101.com/
RE_PATTERN = re.compile("[a-z]+")          

In [6]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [11]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
#!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt
!wget "https://www.gutenberg.org/files/11/11-0.txt" -O data/alice.txt

--2022-01-07 03:19:57--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174313 (170K) [text/plain]
Saving to: `data/alice.txt'

100%[======================================>] 174,313     --.-K/s   in 0.1s    

2022-01-07 03:20:01 (1.29 MB/s) - `data/alice.txt' saved [174313/174313]



In [3]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

﻿The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [8]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [9]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'd'._

> __d)__ This question assumes that we have already ran the wordcount below and produced independent counts for 'hatter' & 'hatters' and 'person' & 'people'. **It would be more efficient at this point to use the outputs of this wordcount to post-process to count the plurals and singulars as one word, since a new tokenizer would need to be run over the entire data versus having much fewer words to go over if already running on the output of wordcount.** 
>
> Additionally, the new tokenizer would still require a regular expression to handle the extra `+'s'` when pluralizing words such as 'hatter' & 'hatters' but it would also need a way of recognizing strange plurazations such as 'person' & 'people'. In our wordcount results below, a grep for 'hatter' yielded 2 lines, 'hatter' and 'hatters', along with their counts. We would simply need to add their counts together in post-processing. We would still need to raise some condition for pluralizations such as 'person' & 'people', but atleast the counting work is completed already. A new tokenizer would still need to go through every sentence to produce counts.


In [ ]:
# part a - DO YOUR WORK IN wordCount.py

In [1]:
!cat wordCount.py

#!/usr/bin/env python
"""
This script reads lines from STDIN and returns a list of
all words and the count of how many times they occurred.

INPUT:
    a text file
OUTPUT FORMAT:
    word \t count
USAGE:
    python wordCount.py < yourTextFile.txt

Instructions:
    Fill in the missing code below so that the script
    prints tab separated word counts to Standard Output.
    NOTE: we have performed the tokenizing for you, please
    don't modify the provided code or you may fail unittests.
"""

# imports
import sys
import re
from collections import defaultdict

counts = defaultdict(int)

# stream over lines from Standard Input
for line in sys.stdin:

    # tokenize
    line = line.strip()
    words = re.findall(r'[a-z]+', line.lower())

############ YOUR CODE HERE #########
    # add words to the counts dict
    for word in words:
        counts[word] += 1
        
# after all lines processed, print out word counts
for k, v in counts.items():
    print(f"{k}\t{v}")




############ (END

In [32]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this	3
is	2
a	2
small	3
test	3
file	3
for	1
has	1
two	1
lines	1


In [22]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [23]:
!head data/alice_counts.txt

the	1839
project	88
gutenberg	98
ebook	13
of	638
alice	403
s	222
adventures	11
in	435
wonderland	7


__Check your results:__ How many times does the word "alice" appear in the book? 

In [24]:
# EXPECTED OUTPUT: 404
!grep alice data/alice_counts.txt

alice	403


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [25]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter	56
hatters	1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [21]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen	76
queens	1


# Question 5: Unix Sorting Practice
Another common task in this course's assignments will be to make strategic use of sorting.     

### Q5 Tasks:
* __a) short response:__ What is the Big O complexity of the fastest comparison based sorting algorithms? [*HINT: If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).*]

* __b) short response:__ What is the default sorting algorithm in MapReduce? What is the Big O complexity of this algorithm? Why do you think this algorithm was chosen? [*HINT: Julius Ceasar! (week 1 slides)*]

* __c) code:__ write a unix command to check how many records are in your word count file.

* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [*HINT: if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)*]

* __e) code:__ Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ The fastest comparison based sorting algorithms can have a Big O time complexity of `O(n)` in the best case, and a Big O time complexity of `O(n log(n))` in the worst case. Their average Big O time complexity is `O(n log(n))`. **Therefore, the Big O complexity is `O(n log(n))`.** [Source](https://www.bigocheatsheet.com/).

> __b)__ The default sorting algorithm in MapReduce is Merge-Sort. It has a Big O time complexity of `O(n log(n))`. It was chosen because it is among the fastest comparison based sorting algorithms and is a divide and conquer algorithm that breaks the sorting problem down and distributes the work to sort our data object.



In [12]:
# part c - write a unix command to check how many records are in your word count file
# source: https://www.thegeekdiary.com/how-to-count-lines-in-a-file-in-unix-linux/
!cat data/alice_counts.txt | wc -l

3006


In [26]:
# part d - unix command to sort your word counts alphabetically 
!sort -k1,1 data/alice_counts.txt > data/alice_counts_A-Z.txt

In [27]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [29]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -n -r -k2,2 data/alice_counts.txt > data/alice_counts_sorted.txt

In [30]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the	1839
and	942
to	811
a	695
of	638
it	610
she	553
i	546
you	486
said	462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1837
and	946
to	811
a	695
of	637
it	610
she	553
i	546
you	487
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ With no reducer specified, we are only doing the mapping operation using the `wordCount.py` python script. As a result, the 4 parallel processes independently count up the instances of 'alice' that they encounter from their line chunks. The total count of 'alice' from earlier, 403, is broken up among these 4 processes and their results. $113 + 126 + 122 + 42 = 403$. To fix this, we need to add a reducer that can aggregate, by summing, these counts of 'alice' into one total count of 403.

In [5]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [6]:
!which bash

/usr/bin/bash


In [7]:
!pwd

/home/anand/w261-s22-Anand-Patel-95/Assignments/HW1-work


In [11]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [12]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	113
alice	126
alice	122
alice	42


In [10]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [2]:
!cat pWordCount_v1.sh

#!/bin/bash
# pWordCount.sh
# Author: James G. Shanahan
# Usage: pWordCount.sh m testFile.txt mapper.py [reducer.py]
# Input:
#   m = number of processes (maps), e.g., 4
#   inputFile = a text input file
#   mapper = an executable that reads from STDIN and prints to STDOUT
#   reducer = (optional) an executable that reads from STDIN and prints 
#             to STDOUT, if no reducer is provided, the framework will
#             simply stream the mapper output.
#
# Instructions:
#    For Q6a - Read this script and its comments closely. Ignore the
#              part marked "Otherwise" in STEP 3, you'll use that later.
#    For Q6c - Add a single line of code under '#Q6c' in STEP 3 so that
#              the script pipes the output of each chunk's word countfiles
#              into the second executable script provided as an argument,
#              Note that we saved the script name (which was the 4th arg)
#              to the variable $reducer. It can be executed by piping the
#     

In [20]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [21]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


In [19]:
!man sort

This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, including manpages, you can run the 'unminimize'
command. You will still need to ensure the 'man-db' package is installed.


# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ This supposedly 'parallelized' Word Count wouldn't work on a really large input corpus because the reducer keeps track of word counts using a dictionary for the entire corpus. The Python object that aggregates our counts (defaultdict) exists in memory. If the vocabulary is too large for the memory space available we would crash the notebook. This is a very memory constrained implementation, that will have trouble scaling to a larger dataset, i.e. a larger corpus and dictionary.

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ Let's assume that we have the outputs of the mappings and sorted them alphabetically such that we have one big list of 'word \t count' that is too large to fit on a single computer. We want to split these sorted word counts in half and send these halves to the reducers. The problem is where the split occurs might be in between the word counts for the same word; let's say the split happens halfway through after a line "the \t 150" and before a line "the \t 90". This results in the same word having counts sent to both reducer 1 and reducer 2. This would lead to the 2 output files post-reducing to both contain word counts for "the". We wanted the word count of "the" to be aggregated in one place. We need each key to be sent to just 1 reducer.

> __e)__ When we split halfway through the data to send to our 2 reducers, we can check if the line before and after the split location have the same key (ex. "the"). If yes, split further down until the key on the lines before and after the split are not the same. This adjusts the split such that each key goes to only one reducer after partitioning. Then we can reduce these partitions separately.

In [22]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.py

In [3]:
!cat pWordCount_v2.sh

#!/bin/bash
# pWordCount.sh
# Author: James G. Shanahan
# Usage: pWordCount.sh m testFile.txt mapper.py [reducer.py]
# Input:
#   m = number of processes (maps), e.g., 4
#   inputFile = a text input file
#   mapper = an executable that reads from STDIN and prints to STDOUT
#   reducer = (optional) an executable that reads from STDIN and prints 
#             to STDOUT, if no reducer is provided, the framework will
#             simply stream the mapper output.
#
# Instructions:
#    For Q7b - Ammend this script in STEP 2 and STEP 3 to 
#              alphabtetically sort the contents of each chunk before
#              piping them into the reducer script and redirecting on to 
# .            $data.output.
# --------------------------------------------------------------------

usage()
{
    echo ERROR: No arguments supplied
    echo
    echo To run use
    echo "pWordCount.sh m inputFile mapper.py [reducer.py]"
    echo Input:
    echo "number of processes/maps, EG, 4"
    echo "mapper.

In [4]:
!cat aggregateCounts_v2.py

#!/usr/bin/env python
"""
This script reads word counts from STDIN and aggregates
the counts for any duplicated words.

INPUT & OUTPUT FORMAT:
    word \t count
USAGE (standalone):
    python aggregateCounts_v2.py < yourCountsFile.txt

Instructions:
    For Q7 - Your solution should not use a dictionary or store anything   
             other than a single total count - just print them as soon as  
             you've added them. HINT: you've modified the framework script 
             to ensure that the input is alphabetized; how can you 
             use that to your advantage?
"""

# imports
import sys


################# YOUR CODE HERE #################
word_prev = "-1"
total_count = 0

# stream over lines from Standard Input
for line in sys.stdin:
    # extract words & counts
    word, count  = line.split()
    # print(f"line: {word} {count}")
   
    if total_count == 0:
        # if this is the first word
        # set the first line to the tracked word & count
        word_prev

In [25]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [26]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [27]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	403


# Question 8: Scalability Considerations
In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ The two features of an ideal algorithm from a scalability perspective are: "First, in terms of data: given twice the amount of data, the same algorithm should take at most twice as long to run, all else being equal. Second, in terms of resources: given a cluster twice the size, the same algorithm should take no more than half as long to run."
>
> Basically the two criteria are that 1) 2x as much data will take at most 2x as long to run, 2) 2x the the number of cores or resources in your cluster should atleast halve your time to run.

> __b)__ The time it takes to run the algorithm very roughly halves when going from 1 --> 2 partitions and then going from 2 --> 4 partitions. As we double the number of cores we use, or partitions we process in parallel, the algorithm run time roughly halves. 
>
> We are running the same sized data in all 3 tests so we can only see how our observations meet Lin and Dyer's 2nd criteria that doubling the number of resources per cluster will cause the algorithm to take no more than half as long to run. This criteria is not exactly met, but is roughly observed, since the run time is not exactly halved each time we double our partitions. From 1 --> 2 partitions, it takes 57% as long. From 2 --> 4, it takes 63% as long. There is some extra overhead that comes with sorting the partitioned data, and for this relatively small dataset this adds noticeably to the run time, which would theoretically be halved by doubling the cores. If we were running the algorithm on a larger dataset, the doubling of cores would result in the run time being closer to being halved.

> __c)__ The run times decrease from 1 --> 2, they do not decrease from 2 --> 4 (staying around the same time as 1 core), and increase from 4 --> 8. I expected the run time to increase going 4 partitions to 8 partitions, since my computer set up only has 4 cores so the overhead of making more than 4 paritions and shuffling around will cost some runtime. Optimally, my machine would run 4 paritions with its 4 cores. 
>
> I expected going from 2 --> 4 partitions to decrease the run time, but it did not here. My theory is that for large datasets, it should theoretically decrease the run time. However, there is some overhead with the implementation, namely sorting the intermediate outputs of the partitions and mappings. The overhead of doing 4 sorting operations vs. 2 sorting operations adds some run time complexity, which leads to the increased time on small datasets where partitioning does not offer much benefit. If this dataset was larger, we would  see more decrease in runtime as we add paritions and use more cores (up to the limit of the computer). This means that our implementation will scale well with large datasets, but does not offer much benefit when working with small datasets.

> __d)__ Adding more partitions adds to the number of mapper processes run, and sorts done on the output of these mappings before going to the reducer. Increasing the number of partitions causes the amount of data sent to each mapper to decrease, therefore decreasing the total amount of time spent during each of these mappings. The tradeoff this costs is the number of sorting operations increasing, since we have additional mapper outputs to now sort.

__Run the following cells to create the mapper referenced in `part b`__

In [55]:
!mkdir demo

mkdir: cannot create directory ‘demo’: File exists


In [ ]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

In [28]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [29]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

5.27 s ± 38.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

3.04 s ± 64.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.94 s ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [32]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

877 ms ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

864 ms ± 32.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

879 ms ± 48.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1.01 s ± 42.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1.22 s ± 85.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

1.79 s ± 93.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Chapter 2 in DITP_ ]

### Q9 Student Answers:

> __a)__ In the context of word counting, an "Embarrassingly Parallel" word counting workload describes the little to no effort needed to separate the problem of counting word frequencies into a number of parallel tasks. In this parallel word counting implementation, the documents of interest have their lines chunked up and sent to parallel tasks that will count word frequencies, with no dependency or need for communication between these parallel tasks.
>
> In our implementation in this notebook, we counted words in parallel through 4 mapper processes running on chunks of lines from Alice in Wonderland, and aggregated the results into one output document. These parallel word counting tasks did not require any communication between each other or any effort to pre or post-process the results of the counts (aside from sorting the outputs indiscriminately to improve the memory efficiency of the reducer). This was an embarrassingly parallel implementation of the word counting problem.

> __b)__ Type your (OPTIONAL) answer here!

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform